<a href="https://colab.research.google.com/github/RutgerEttes/PubMedBERT-Question-Classification/blob/main/question_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch===1.7.1+cu110 torchvision===0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install deepspeed
!pip install mpi4py

In [ ]:
import transformers
import pandas as pd
import numpy as np
import sklearn
import csv
import matplotlib.pyplot as plt
from collections import Counter
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
!cp /content/drive/MyDrive/LSDP\ Project/Notebooks/RQ3_utils.py .
import RQ3_utils
from deepspeed.utils import logger
logger.setLevel(50)

In [ ]:
ichi_train_path = '/content/drive/MyDrive/LSDP Project/data/ICHI2016-TrainData.tsv'
ichi_test_path = '/content/drive/MyDrive/LSDP Project/data/new_ICHI2016-TestData_label.tsv'
icliniq_annotated_path = '/content/drive/MyDrive/LSDP Project/data/icliniq_annotated.tsv'
COVID_annotated_path = '/content/drive/MyDrive/LSDP Project/data/COVID_annotated.tsv'
icliniq_path = '/content/drive/MyDrive/LSDP Project/data/dataicliniq_dialogue_questions.tsv'
COVID_path = '/content/drive/MyDrive/LSDP Project/data/dataCOVID-Dialogue-Dataset-English_questions.tsv'

In principle all datasets have been prepared to for use in the prepare_datasets function, but should you want to create *dataCOVID-Dialogue-Dataset-English_questions.tsv* and *dataicliniq_dialogue_questions.tsv* from scratch, simply run the following function with the desired input and output filepaths:


```
preprocess_to_tsv(data_path, save_to)
```



In [ ]:
filepaths = [ichi_train_path,
ichi_test_path,
icliniq_annotated_path,
COVID_annotated_path,
icliniq_path,
COVID_path]

train_dataset, val_dataset, test_dataset, COVID_annotated_dataset, icliniq_annotated_dataset, COVID_dataset, icliniq_dataset = RQ3_utils.prepare_datasets(filepaths)

In [ ]:
training_args = TrainingArguments(
    output_dir='RQ3/results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='RQ3/logs',            # directory for storing logs
    local_rank=0,
    deepspeed='/content/drive/MyDrive/LSDP Project/Notebooks/ds_config.json',
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    eval_steps=100,
    metric_for_best_model= 'accuracy',
    logging_steps = 100,
    save_total_limit=2
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/LSDP Project/Notebooks/model', num_labels=7).to('cuda')

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=RQ3_utils.compute_metrics
)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", num_labels=7)

# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset,             # evaluation dataset
#     compute_metrics=RQ3_utils.compute_metrics
# )

# trainer.train()

In [ ]:
test_predictions, _, test_metrics = trainer.predict(test_dataset)
COVID_annotated_predictions, _, test_metrics = trainer.predict(COVID_annotated_dataset)
icliniq_annotated_predictions, _, test_metrics = trainer.predict(icliniq_annotated_dataset)
COVID_prediction, _, _ = trainer.predict(COVID_dataset)
icliniq_prediction, _, _ = trainer.predict(icliniq_dataset)

test_pred_labels = [i.argmax() for i in test_predictions]
COVID_annotated_pred_labels = [i.argmax() for i in COVID_annotated_predictions]
icliniq_annotated_pred_labels = [i.argmax() for i in icliniq_annotated_predictions]
COVID_pred_labels = [i.argmax() for i in COVID_prediction]
icliniq_pred_labels = [i.argmax() for i in icliniq_prediction]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
make_misclassification_heatmap(COVID_annotated_dataset.labels, COVID_annotated_pred_labels)

In [ ]:
make_misclassification_heatmap(icliniq_annotated_dataset.labels, icliniq_annotated_pred_labels)